# Scientific Computing 5: Molecular dynamics (MD) simulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import cm
from tqdm import tqdm

### Initalization

In [ ]:
# Physical parameters:
N = 100  # number of particles
L = np.array([10, 10, 10])  # box size row vector [reduced units]
r_min = 0.9  # minimum distance between particles [reduced units]

# Simulation parameters:
h = 0.001  # time step size of the simulation [reduced units]
t_end = 50  # total time of the simulation [reduced units]

# Animation settings:
duration = 10  # video length [s]
FPS = 30  # video framerate [fps]

frames = duration * FPS  # total number of frames
steps_per_frame = int(round(t_end/h/frames))  # timesteps per frame

### Classes and functions

In [ ]:
class Particle:
    def __init__(self):
        self.x = np.zeros(3)  # position
        self.v = np.zeros(3)  # velocity
        self.a = np.zeros(3)  # acceleration (= force in reduced units)

        self.c = np.zeros(3)  # cell index

    def clear_acceleration(self):
        self.a = np.zeros(3)
    
    def make_step(self):  # using semi-implicit Euler method
        self.v = self.v + h * self.a
        self.x = self.x + h * self.v

        self.K = 1/2 * np.linalg.norm(self.v)**2  # calculate kinetic energy
    
    def restrict_to_box(self):
        self.x = self.x - np.floor(self.x / L) * L

### Main code

In [ ]:
particles = [Particle() for i in range(N)]

i = 0

while i < N:  # randomly place all particles in the box
    particles[i].x = np.random.default_rng().uniform(size=3) * L
    for j in range(i-1):
        dx = particles[j].x - particles[i].x
        dx = dx - np.round(dx / L) * L
        if np.linalg.norm(dx) < r_min:  # check particle distance
            i -= 1  # assign new location to particle if it is too close
            break

    i += 1

### Plot initialization

In [ ]:
fig = plt.figure(figsize=(12, 6), dpi=100)

ax1 = fig.add_subplot(1, 2, 1, projection="3d")

ax1.set_xlim(0, L[0])
ax1.set_ylim(0, L[1])
ax1.set_zlim(0, L[2])

ax1.set_title("Molecular dynamics simulation of particles")
ax1.set_xlabel("x [reduced units]")
ax1.set_ylabel("y [reduced units]")
ax1.set_zlabel("z [reduced units]")

dots = []
for _ in range(N):
    dot = ax1.scatter([], [], [], marker="o")
    dots.append(dot)

ax2 = fig.add_subplot(1, 2, 2)

ax2.set_xlim(0, t_end)
ax2.set_ylim(0, 1)
ax2.set_title("Temperature of the simulated particle system over time")
ax2.set_xlabel("Time [reduced units]")
ax2.set_ylabel("Temperature [reduced units]")

lt, = ax2.plot([], [], color="k")

data = [dots, lt]
t, T = [], []

plt.tight_layout()
plt.subplots_adjust(wspace=0.2)

### Run the simulation

In [ ]:
def run_simulation(frame):
    for step in range(steps_per_frame):
        for i in range(N):
            # Clear particle forces before calculating new interactions:
            particles[i].clear_acceleration()

        for i in range(N):
            for j in range(i-1):
                # Calculate particle distance (minimum image convention):
                dx = particles[j].x - particles[i].x
                dx = dx - np.round(dx / L) * L
                r = np.linalg.norm(dx)  # calculate distance

                # Calculate Lennard-Jones force in reduced units:
                f = 24/r * (2/r**12 - 1/r**6) * (dx / r)

                # Update forces between interacting particles:
                particles[i].a = particles[i].a - f
                particles[j].a = particles[j].a + f

        for i in range(N):
            particles[i].make_step()  # make time step
            particles[i].restrict_to_box()  # move particles to the box again

            # Update the position of the 3D scatter plot element:
            data[0][i]._offsets3d = ([particles[i].x[0]], [particles[i].x[1]], [particles[i].x[2]])

            velo = np.linalg.norm(particles[i].v)
            norm_velo = velo / np.max([np.linalg.norm(x.v) for x in particles])

            data[0][i].set_facecolors(cm.winter(norm_velo))  # set color according to color map

    t.append(round(frame * steps_per_frame * h, 2))
    T.append(2 * sum(x.K for x in particles) / (3 * (N-1)))

    data[1].set_data(t, T)

    progress.update(1)

    return data

### Save the simulation animation

In [ ]:
progress = tqdm(total=frames+1)
ani = FuncAnimation(fig, run_simulation, frames=frames, interval=1000/FPS)
ani.save(f"output/md_naive_{N}_{FPS}fps.mp4", fps=FPS)
progress = tqdm(total=frames+1)